# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открытие данных и общее изучение полученной информации

In [33]:
import pandas as pd
from pymystem3 import Mystem                # подготовим для лемматизации
m = Mystem()
data = pd.read_csv('/datasets/data.csv')   # Скачаем файл в data

display('Размер таблицы: ',data.shape)
display('Информация по данным')
display(data.info())                                # Посмотрим на данные
display()
display(' Собственно таблица')

display(data.head(20))

'Размер таблицы: '

(21525, 12)

'Информация по данным'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

' Собственно таблица'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Полученные данные представляют собой таблицу размером 21 524 строки, плюс заголовки и 12 колонок данных.

int64    -  dob_years, education_id, family_status_id, debt

float64  -  days_employed, total_income

object   - education, family_status, gender, income_type, purpose


Пропуски данных имеются в days_employed и total_income. Возможно, что эти пропуски связаны между собой, так как их количество совпадает.

Проблемы в данных:

days_employed (стаж работы) имеетсся отсутствие данных NaN  просто некоректные данные в виде больших положительных чисел

education (образование) написано в различных регистрах

education_id представлено в виде int64 . В первом приближении по информационному наполнению эквивалентно 'education'

family_status  надо будет разобраться с пробелами и выравниванием

family_status_id представлено в виде int64. В первом приближении по информационному наполнению эквивалентно 'family_status'

gender возможны лишние  пробелы 

incom_type (вид занятости) возможны пробелы пробелы 

debt (признак наличия отсутствия прежних задолженностей)  - значение 1 или 0

total_income (общий доход) имеет отсутствие данных NaN

purpose (цель кредита)  возможны пробелы


Имеет смысл посмотреть на таблицу с точки зрения заданных вопросов. Спрашивают следующие зависимости:
 - наличие детей и возврат кредита в срок
 - семейное положение и возврат кредита в срок
 - уровень дохода и возврат кредита в срок
 - цель кредита и возврат кредита в срок
 
 
 Таким образом для нас важна полнота следующих значений в данных:
 - children количество детей
 - family_status или family_status_id семейный статус или его идентификатор. Возможно, они дублируют друг-друга
 - total_income общий доход заемщика
 - purpose цель кредита
 - debt история с задолженностью
 
 Остальные данные пока можно рассматривать как не нужные или как данные, которые могут быть использованы при уточнении информации по необходимым столбцам.
 
 




## Шаг 2. Предобработка данных


### Обработка пропусков

Для начала проверим интересующие нас данные на наличие чего-либо необычного. Используем метод .unique()



In [34]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


**Неожиданно!**
Если с четырьмя и пятью детьми еще можно мириться, то с -1 и 20 не стоит. Посмотрим на эти строки

In [35]:
display(data[(data.loc[:,'children'] == 20)]) #строки со значение детей 20
#    было     print(data.loc[(data.loc[:,'children'] == 20)]) #строки со значение детей 20
display(data[(data.loc[:,'children'] == -1)]) # аналогично посмотрим на -1
#    было    print(data.loc[(data.loc[:,'children'] == -1)]) # аналогично посмотрим на -1

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


Ничего особенного и подозрительного не обнаружено. Можно предположить, что при наборе данных вместо 2 детей ввели 20, а вместо 1 ввели -1

Посмотрим на общее количество таких данных. Просто методом .loc строки с нужными нам значениями и методом .count посчитаем их

In [36]:
print('Записей с количеством детей 20 ', data.loc[(data.loc[:,'children'] == 20)]['children'].count())
print('Записей с количеством детей -1 ', data.loc[(data.loc[:,'children'] == -1)]['children'].count())

Записей с количеством детей 20  76
Записей с количеством детей -1  47


В общем массиве данных 76 и 47 записей составляют 0,35 и 0,21 % соответственно. Без большого вреда для данных, 
их можно просто удалить или заменить в соответствии со сделанными выше предположениями.

Сделаем замену, заменим 20 на 2 и -1 на 1. Для доступа и замены используем метод .loc

Методом .unique проверим, какие значения остались после замены в столбце количество детей.

In [37]:
data.loc[(data.loc[:,'children'] == -1), 'children'] = 1
#data.loc[(data.loc[:,'children'] == -1),['children']] = 1
data.loc[(data.loc[:,'children'] == 20),'children'] = 2
display('Возможное количество детей ', data['children'].unique())

'Возможное количество детей '

array([1, 0, 3, 2, 4, 5])

Количество детей приведено в порядок

Посмотрим семейное положение методом .unique

In [38]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Пять значений, охватывающих семейное положение. Ничего лишнего.

Можно проверить, соответствует ли столбец 'family_status' столбцу 'family_status_id'.


print('Несовпадений женат/замужем со значением 0: ',
     data.loc[(data.loc[:,'family_status'] == 'женат / замужем') & 
         (data.loc[:,'family_status_id'] != 0)]['family_status'].sum()
     )
print('Несовпадений гражданский брак со значением 1: ',
     data.loc[(data.loc[:,'family_status'] == 'гражданский брак') & 
         (data.loc[:,'family_status_id'] != 1)]['family_status'].sum()
     )
print('Несовпадений вдовец / вдова со значением 2: ',
     data.loc[(data.loc[:,'family_status'] == 'вдовец / вдова') & 
         (data.loc[:,'family_status_id'] != 2)]['family_status'].sum()
     )
print('Несовпадений в разводе со значением 3: ',
     data.loc[(data.loc[:,'family_status'] == 'в разводе') & 
         (data.loc[:,'family_status_id'] != 3)]['family_status'].sum()
     )
print('Несовпадений Не женат / не замужем со значением 4: ',
     data.loc[(data.loc[:,'family_status'] == 'Не женат / не замужем') & 
         (data.loc[:,'family_status_id'] != 4)]['family_status'].sum()
     )


In [39]:
data.groupby('family_status')['family_status_id'].unique()

family_status
Не женат / не замужем    [4]
в разводе                [3]
вдовец / вдова           [2]
гражданский брак         [1]
женат / замужем          [0]
Name: family_status_id, dtype: object

Таким образом, информация в столбцах  'family_status' столбцу 'family_status_id' полностью дублирована и один из них можно будет
исключить из рассмотрения


Продолжим с debt.
Пропусков информации, как мы знаем, нет. Значение debt должно быть либо 0, либо 1. Проверим на содержимое


In [40]:
data['debt'].unique()

array([0, 1])

Все в норме.  Значение debt либо 0, либо 1.

Просмотрим показатели дохода. В первую очередь известно, что в них масса пропусков. Посмотрим строки, в которых пропуски. Возможно, 
в них будут какие-то закономерности.

In [41]:
print(data[data['total_income'].isna()].head(20))
print('Количество строк с пропусками: ',len(data[data['total_income'].isna()]))

     children  days_employed  dob_years education  education_id  \
12          0            NaN         65   среднее             1   
26          0            NaN         41   среднее             1   
29          0            NaN         63   среднее             1   
41          0            NaN         50   среднее             1   
55          0            NaN         54   среднее             1   
65          0            NaN         21   среднее             1   
67          0            NaN         52    высшее             0   
72          1            NaN         32    высшее             0   
82          2            NaN         50    высшее             0   
83          0            NaN         52   среднее             1   
90          2            NaN         35    высшее             0   
94          1            NaN         34    высшее             0   
96          0            NaN         44   СРЕДНЕЕ             1   
97          0            NaN         47    высшее             

Пропущено 2174 строки из 21525 строк данных. Это практически 10%. Судя по анализу данных, закономерность видна только одна - там,
где не прописан стаж, там нет и дохода. Возможно, данные были сгруппированы из разных источников. Возможны два варианта обработки пропусков - заполнить средним значением или пропустить их, удалить.

Принимаем решение заменить пропуски на среднее

Вычислим среднее по столбцу методом .mean и заменим на него все отсутствующие выражения методом .fillna

In [42]:
mean = data['total_income'].mean()
print('Средний доход по столбцу', mean)
data['total_income'] = data['total_income'].fillna(mean)  #.fillna(data['total_income'].mean())
print(data.head(15))




Средний доход по столбцу 167422.30220817294
    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887     

В 12-ой строке total_income  было NaN, стало среднее значение. Метод отработал корректно. Еще проверим на наличие пропусков методом .info

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски отсутствуют. Можно сделать вывод, что замена произведена.

Проверим, нет ли каких либо проблем в столбце 'debt'. В нем должны быть только ноли и единицы.

In [44]:
data['debt'].unique()

array([0, 1])

Так оно и оказалось. Делать с этим столбцом ничего не надо.



### Вывод

Мы сделали замену некорректных данных в столбце children, заменили на среднее значение отсутствующие данные в 'total_income'. На этом этапе считаем, что можно преобразовывать и оптимизировать данные.

### Замена типа данных

В данных в целях экономии места можно заменить total_income типа float64 на тип int64

Используем метод .astype с аргументов int


In [45]:
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Видно, что total_income        21471 non-null int64

### Вывод

В total_income убрали несущественную информацию в виде дробных частей дохода


### Обработка дубликатов

Посчитаем дубликаты методом .duplicated  в паре с .sum и посмотрим на них.

In [46]:
print('Всего дубликатов',data.duplicated().sum())
print()
print('Дубликаты')
print(data.duplicated().value_counts())
print()
data.loc[data.duplicated()]

Всего дубликатов 54

Дубликаты
False    21471
True        54
dtype: int64



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,167422,покупка жилья для семьи
4182,1,NaN,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,167422,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,167422,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,167422,сыграть свадьбу
7808,0,NaN,57,среднее,1,гражданский брак,1,F,пенсионер,0,167422,на проведение свадьбы
8583,0,NaN,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,167422,дополнительное образование
9238,2,NaN,34,среднее,1,женат / замужем,0,F,сотрудник,0,167422,покупка жилья для сдачи
9528,0,NaN,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,167422,операции со своей недвижимостью
9627,0,NaN,56,среднее,1,женат / замужем,0,F,пенсионер,0,167422,операции со своей недвижимостью
10462,0,NaN,62,среднее,1,женат / замужем,0,F,пенсионер,0,167422,покупка коммерческой недвижимости


В таблице оказалось 54 дублированнх записи. Просомтр их с использованием метода value_counts ничего особенного не выявил. Удаляем, используя метод .drop_duplicates а затем восстанавливаем индексацию методом .reset_index(drop=True)

In [47]:
data = data.drop_duplicates().reset_index(drop=True) 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
children            21471 non-null int64
days_employed       19351 non-null float64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Было 21525 строк, стало 21471.


Количество строк с данными стало на 54 меньше за счет удаления дубликатов.


### Вывод

По всем интересующим нас данным, кроме целей кредита, устранены пропуски и дубликаты.

### Лемматизация

Разберемся с целью кредита. 
Для начала посмотрим, на какие цели брался кредит

In [48]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Много чего написано, но можно выделить основное:

- автомобиль
- образование
- свадьба
- недвижимость
Последний пункт можно было бы проанализировать на предмет коммерческой и жилой недвижимости, но в ряде записей не хватает информации для этого. Потому оставляем просто "недвижимость" любого типа. 

Для дальнейшей работы с этим столбцом проведем лематизацию данных, то есть приведение всех слов в лемме. Затем проведем анализ каждой строки на ее содержимое и сделаем замену строк: оставим только четыре значения: автомобиль, образование, свадьба, недвижимость.

Будем использовать лемматизацию меодом pymystem3. Вызов библиотеки поставим в верхние строчки программы.



In [49]:

for i in range(len(data)):
    a =  data.loc[i,'purpose']
    b = ",".join(m.lemmatize(a))
    data.loc[i,'purpose'] = b
print('Результат замены')
data.head()



Результат замены


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"покупка, ,жилье,\n"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"приобретение, ,автомобиль,\n"
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,"покупка, ,жилье,\n"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"дополнительный, ,образование,\n"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"сыграть, ,свадьба,\n"


Теперь можем заменять на однословные выражения

В каждой записи проверим ключевое слово (автомобиль, образование, свадьба и жилье с недвижимостью) и заменим на новые значения: "Покупка автомобиля, "Оплата образования", "На свадьбу", "Недвижимость"

In [50]:
# for i in range(len(data)):
#     if 'автомобиль' in data.loc[i,'purpose']:
#         data.loc[i,'purpose'] = 'Покупка автомобиля'
#     if 'образование'in data.loc[i,'purpose']:
#         data.loc[i,'purpose'] = 'Оплата образования'
#     if 'свадьба'in data.loc[i,'purpose']:
#         data.loc[i,'purpose'] = 'На свадьбу'
#     if ('жилье' in data.loc[i,'purpose']) or ('недвижимость' in data.loc[i,'purpose']):
#         data.loc[i,'purpose'] = 'Недвижимость'
# data.head()


In [51]:
def replacement_target(target_colums):
    if 'автомобиль' in target_colums:
        return 'Покупка автомобиля'
    if 'образование'in target_colums:
        return 'Оплата образования'
    if 'свадьба'in target_colums:
        return 'На свадьбу'
    if ('жилье' in target_colums) or ('недвижимость' in target_colums):
        return 'Недвижимость'
print(replacement_target('образование'))

data['purpose'] = data['purpose'].apply(replacement_target) 

data.head()

Оплата образования


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,Недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,Покупка автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,Недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,Оплата образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,На свадьбу


Проверим, все ли замены сделаны

In [52]:
data['purpose'].unique()

array(['Недвижимость', 'Покупка автомобиля', 'Оплата образования',
       'На свадьбу'], dtype=object)

### Вывод

В данных осталось лишь четыре цели получения кредита. Что мы и хотели получить

### Подготовка данных к анализу

Для начала уберем лишние столбцы из data.  Нам не нужны следующие данные: days_employed , dob_years , education , gender , income_type . Воспользуемся методом .drop  с аргументом axis = 1

In [53]:
data = data.drop('days_employed', axis = 1)
data = data.drop('dob_years', axis = 1)
data = data.drop('education', axis = 1)
data = data.drop('education_id', axis = 1)
data = data.drop('gender', axis = 1)
data = data.drop('income_type', axis = 1)
data.head()

,children,family_status,family_status_id,debt,total_income,purpose
0,1,женат / замужем,0,0,253875,Недвижимость
1,1,женат / замужем,0,0,112080,Покупка автомобиля
2,0,женат / замужем,0,0,145885,Недвижимость
3,3,женат / замужем,0,0,267628,Оплата образования
4,0,гражданский брак,1,0,158616,На свадьбу


Получилась не замутненная избыточными данными таблица.

Начнем с подготовки итоговых данных по детям. Для этого сгруппируем данные по количеству детей и в каждой категории посчитаем количество записей по детям, а затем сумму по столбцу debt. Так как он содержит 1 в случае задержек, то получим общее число задержек по каждому из возможных количеств детей.

Затем просто поделим количество задержек по каждой категории на общее количество записей и разместим информацию в переменную total_chidren


In [54]:
#children_grouped_all = data.groupby('children').count()
#display(children_grouped_all)
#children_grouped_debt = data.groupby('children').sum()
#display(children_grouped_debt)
#total_chidren = children_grouped_debt['debt'] / children_grouped_all['debt']
#display(total_chidren)
total_children = data.groupby('children').mean()
total_children = total_children.drop('family_status_id', axis = 1)
total_children = total_children.drop('total_income', axis = 1)
display(total_children)

,debt
children,
0,0.075353
1,0.091639
2,0.094925
3,0.081818
4,0.097561
5,0.000000


По семейному статусу все аналогично детям, результа в переменной total_family_status

In [55]:
family_status_grouped_all = data.groupby('family_status').count()
display(family_status_grouped_all)
family_status_grouped_debt = data.groupby('family_status').sum()
display(family_status_grouped_debt)
total_family_status = family_status_grouped_debt['debt'] / family_status_grouped_all['debt']
display(total_family_status)

,children,family_status_id,debt,total_income,purpose
family_status,,,,,
Не женат / не замужем,2810,2810,2810,2810,2810
в разводе,1195,1195,1195,1195,1195
вдовец / вдова,959,959,959,959,959
гражданский брак,4163,4163,4163,4163,4163
женат / замужем,12344,12344,12344,12344,12344


,children,family_status_id,debt,total_income
family_status,,,,
Не женат / не замужем,654,11240,274,472767290
в разводе,519,3585,85,202788684
вдовец / вдова,151,1918,63,139995608
гражданский брак,1921,4163,388,694802809
женат / замужем,7066,0,931,2084359534


family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093202
женат / замужем          0.075421
Name: debt, dtype: float64

**Вывод**

Аналогично с целями кредита, результат в переменной purpose_status

In [56]:
purpose_grouped_all = data.groupby('purpose').count()
display(purpose_grouped_all)
purpose_grouped_debt = data.groupby('purpose').sum()
display(purpose_grouped_debt)
purpose_status = purpose_grouped_debt['debt'] / purpose_grouped_all['debt']
display(purpose_status)

,children,family_status,family_status_id,debt,total_income
purpose,,,,,
На свадьбу,2335,2335,2335,2335,2335
Недвижимость,10814,10814,10814,10814,10814
Оплата образования,4014,4014,4014,4014,4014
Покупка автомобиля,4308,4308,4308,4308,4308


,children,family_status_id,debt,total_income
purpose,,,,
На свадьбу,1096,2335,186,387483369
Недвижимость,5229,10486,782,1825028381
Оплата образования,1952,3824,370,661319359
Покупка автомобиля,2034,4261,403,720882816


purpose
На свадьбу            0.079657
Недвижимость          0.072314
Оплата образования    0.092177
Покупка автомобиля    0.093547
Name: debt, dtype: float64

Теперь с уровнем дохода. Для начала найдем минимальный и максимальный доходы.

In [57]:
total_income_min = data['total_income'].min()
print('Минимальный доход, указанный в выборке:',total_income_min)
total_income_max = data['total_income'].max()
print('Максимальный доход, указанный в выборке:',total_income_max)

Минимальный доход, указанный в выборке: 20667
Максимальный доход, указанный в выборке: 2265604


Разброс получился в 100 раз. Для анализа возьмем диапазоны с удвоением 0-50-100-200-400-800-1600-3200 тысяч рублей.

Для начала создадим DataFrame категорий

In [58]:
data_range = {'volues_min_max':['0-50','50-100','100-200','200-400','400-800',
                                    '800-1600','1600-3200'],
                  'range_low':[0,50000,100000,200000,400000,800000,1600000],
                  'range_high':[50000,100000,200000,400000,800000,1600000,3200000]}
range_of_values = pd.DataFrame(data_range)
display(range_of_values)
range_of_values.info()

,volues_min_max,range_low,range_high
0,0-50,0,50000
1,50-100,50000,100000
2,100-200,100000,200000
3,200-400,200000,400000
4,400-800,400000,800000
5,800-1600,800000,1600000
6,1600-3200,1600000,3200000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
volues_min_max    7 non-null object
range_low         7 non-null int64
range_high        7 non-null int64
dtypes: int64(2), object(1)
memory usage: 296.0+ bytes


То, что и хотелось.

Напишем функцию относящую каждую запись в определенную категорию и проверим ее

In [59]:
def range_definition(total_incom_columns):
    if 0 < total_incom_columns <= 50000:
        return '0-50'
    if 50000 < total_incom_columns <= 100000:
        return '50-100'
    if 100000 < total_incom_columns <= 200000:
        return '100-200'
    if 200000 < total_incom_columns <= 400000:
        return '200-400'
    if 400000 < total_incom_columns <= 800000:
        return '400-800'
    if 800000 < total_incom_columns <= 1600000:
        return '800-1600'
    if 1600000 < total_incom_columns <= 3200000:
        return '1600-3200'
    
print(range_definition(30000))
print(range_definition(70000))
print(range_definition(150000))
print(range_definition(250000))
print(range_definition(500000))
print(range_definition(900000))
print(range_definition(3000000))

0-50
50-100
100-200
200-400
400-800
800-1600
1600-3200


Функция работает верно

Применим ее к данным. Для этого в данные добавим столбец incom_range  и заполним ее при помощи метода .applay передав ей в качестве аргумента функцию range_definition, применив его к столбцу total_incom



In [60]:
data['incom_range'] = data['total_income'].apply(range_definition)
display(data)

,children,family_status,family_status_id,debt,total_income,purpose,incom_range
0,1,женат / замужем,0,0,253875,Недвижимость,200-400
1,1,женат / замужем,0,0,112080,Покупка автомобиля,100-200
2,0,женат / замужем,0,0,145885,Недвижимость,100-200
3,3,женат / замужем,0,0,267628,Оплата образования,200-400
4,0,гражданский брак,1,0,158616,На свадьбу,100-200
...,...,...,...,...,...,...,...
21466,1,гражданский брак,1,0,224791,Недвижимость,200-400
21467,0,женат / замужем,0,0,155999,Покупка автомобиля,100-200
21468,1,гражданский брак,1,1,89672,Недвижимость,50-100
21469,3,женат / замужем,0,1,244093,Покупка автомобиля,200-400


In [61]:
data['incom_range'].value_counts()

100-200      11942
200-400       4538
50-100        4091
400-800        485
0-50           372
800-1600        38
1600-3200        5
Name: incom_range, dtype: int64

In [67]:
# Код ревьюера
data['incom_range'] = data['incom_range'].astype('category').cat.reorder_categories(['0-50','50-100','100-200','200-400','400-800','800-1600','1600-3200'], ordered=True)

Категории добавлены в таблицу. 

Осталось провести подготовку итоговой таблицы по категориям доходов, аналогично произведенному выше. Итоговые данные будут в переменной incom_range_status

In [68]:
incom_range_grouped_all = data.groupby('incom_range').count()
display(incom_range_grouped_all)
incom_range_grouped_debt = data.groupby('incom_range').sum()
display(incom_range_grouped_debt)
incom_range_status = incom_range_grouped_debt['debt'] / incom_range_grouped_all['debt']
display(incom_range_status)

,children,family_status,family_status_id,debt,total_income,purpose
incom_range,,,,,,
0-50,372,372,372,372,372,372
50-100,4091,4091,4091,4091,4091,4091
100-200,11942,11942,11942,11942,11942,11942
200-400,4538,4538,4538,4538,4538,4538
400-800,485,485,485,485,485,485
800-1600,38,38,38,38,38,38
1600-3200,5,5,5,5,5,5


,children,family_status_id,debt,total_income
incom_range,,,,
0-50,161,273,23,15582771
50-100,1840,4005,331,325142869
100-200,5761,11803,1029,1776161263
200-400,2266,4334,327,1187748465
400-800,251,456,27,240836321
800-1600,30,29,3,39623177
1600-3200,2,6,1,9619059


incom_range
0-50         0.061828
50-100       0.080909
100-200      0.086166
200-400      0.072058
400-800      0.055670
800-1600     0.078947
1600-3200    0.200000
Name: debt, dtype: float64

Теперь все данные подготовлены к анализу.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Вывод**

Из полученных данных видно, самые ндисциплинированные плательщики это бездетные заемщики.Говорить о том, что самыми дисциплинрованными являются заемщики с пятью детьми, скорее всего не корректно, выборка по ним мала. Всего 9 случаев будет мало для уверенной статистики.
Высокий уровень невозврата в семьях с четырьмя детьми тоже может быть статистической погрешностью, в данных таких семей всего 41.
Разброс параметра составляет 1,29 раз.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Вывод**

Яркой зависимости нет, но можно сделать вывод, что одинокие люди с жизненным опытом (вдовые и разведенные) возвращают кредиты лучше других категорий заемщиков. Разброс параметра составляет 1,29 раз

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод**

Из рассмотрения необходимо откинуть группу с доходом более 1,6 млн рублей. Их всего в категории 5 человек и данные по ним не могут рассматриваться как статистически верные. Самые лучшие плательшики попадают в среднюю категорию 400-800 тысяч рублей 5,5% неплательщиков и самую низкодоходную категорию до 50 тысяч рублей 6,1%. Наихудщие плательщики - владельцы дохода в диапазоне 100-200 тысяч рублей. Среди них задержки платежей в 1,56 раз больше, чем среди категории 400-800

- Как разные цели кредита влияют на его возврат в срок?

**Вывод**

Свадьба и покупка недвижимости являются целями, при которых возврат кредитов несколько выше, чем на доругие цели. В принципе, это два серьезных шага в жизни людей и возможно, они серьезней к этому относятся.
Разброс между минимальным и максимальным значением составляет 1,29 раз.  

## Шаг 4. Общий вывод

**Мы рассмотрели** четыре показателя влияющих на возвратность кредита. Наибольшее влияние на показатели возвратности оказывает уровень дохода заемщика. В зависимости от дохода уровень задержек колеблется по казным категориям доходов в 1,56 раза, при уровне колебаний по всем остальным 1,29 раз. На мой взгляд, делать выводы из каждого показателя отдельно было бы не верно. Имеет смысл рассматривать некоторые зависимости, например пары доход/количество детей или даже тройки семейное положение/доход/количество детей. Такие зависимости были бы более информативны.
